In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define dataset path
dataset_path = "F:\\AIVAR FINAL\\parkinson"

# Define categories
categories = {
    "F_Con": 0,  # Female Control
    "F_Dys": 1,  # Female Dysarthric
    "M_Con": 2,  # Male Control
    "M_Dys": 3   # Male Dysarthric
}

# Function to extract features from a WAV file
def extract_features(file_path, max_length=500):
    try:
        audio, sr = librosa.load(file_path, sr=22050)  # Load audio file
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)  # Extract MFCC features
        mfccs = np.mean(mfccs.T, axis=0)  # Compute mean over time axis
        return mfccs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Prepare dataset
features = []
labels = []

for category, label in categories.items():
    folder_path = os.path.join(dataset_path, category)
    
    for file in os.listdir(folder_path):
        if file.endswith(".wav"):  # Ensure file is a WAV file
            file_path = os.path.join(folder_path, file)
            mfcc_features = extract_features(file_path)
            
            if mfcc_features is not None:
                features.append(mfcc_features)
                labels.append(label)

# Convert to NumPy arrays
X = np.array(features)
y = np.array(labels)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a simple Neural Network
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 categories
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


ImportError: cannot import name 'keras' from 'tensorflow' (unknown location)